In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import os
import re

In [77]:
def html_to_txt(file):
    
    # read file
    with open(file, encoding='utf-8', errors='ignore') as f: html_content = f.read()
    
    # remove credits paragraph (first p tag)
    first_p_start_index = html_content.find('<p>')
    first_p_end_index = html_content.find('</p>', first_p_start_index) + len('</p>')
    html_content = html_content[:first_p_start_index] + html_content[first_p_end_index:]

    html_content = re.sub('<font.*?>', '', html_content, flags=re.DOTALL)
    html_content = re.sub('</font>', '', html_content)
    html_content = re.sub('<b>', '', html_content)
    html_content = re.sub('</b>', '', html_content)
    html_content = re.sub('<strong>', '', html_content)
    html_content = re.sub('</strong>', '', html_content)
   
    # initialize soup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # remove title
    for title in soup.find_all('title'): title.extract()
        
    # replace newlines in dialogous with spaces
    for p in soup.find_all('p'):
        p.string = p.get_text().replace('\n', ' ')
        while p.get_text().count('  ') > 0: p.string = p.get_text().replace('  ', ' ')
    
    # get script, lowercase it
    text = soup.get_text()

    # splitlines, join all lines, lowercase, remove weird space, replace . . . with ... for consistency
    text = '\n'.join(line.strip() for line in text.splitlines() if line.strip()).lower().replace('   ', ' ').replace('. . .', '...')
    # remove "end" from end, remove credits
    text = text.rstrip('the end').rstrip('end').replace('opening credits', '').replace('closing credits', '').replace('ending credits', '').replace('end credits.', '')
    # remove transcriber's note
    text = re.sub('{transcriber.+}', '', text)

    if text.count('\n')<50: 
        # add newlines before dialogue start (issue in some files)
        text = re.sub(r' (?=[a-z]+:)', r'\n', text)
        # add newlines before scene descriptions (issue in some files)
        s = re.sub(r' +(\[.*?\])\n', r'\n\1\n', text)
        while s != text:
            text = s
            s = re.sub(r' (\[.*?\])\n', r'\n\1\n', text)
    # add start and end tokens
    text = f'<EPISODE_START>\n{text}\n<EPISODE_END>'
    # remove unnecesary new lines
    text = re.sub(r'\n\n+', 'r\n', text)
    # remove unnecessary spaces
    text = re.sub(r'  +', r' ', text)
    # remove everything between start token and episode name
    text = re.sub(r'<EPISODE_START>\n.*?the one', r'<EPISODE_START>\nthe one', text, flags=re.DOTALL)
    # replace shorter names which are in some files
    text = text.replace('chan: ', 'chandler: ').replace('mnca: ', 'monica: ').replace('phoe: ', 'phoebe: ').replace('rach: ', 'rachel: ')
    
    return text

In [78]:
# convert all .html to .txt
read_dir = './data/raw/scripts'
write_dir = './data/interim/scripts'
for filename in os.listdir(read_dir):
    with open(f'{write_dir}/{filename.replace('.html','.txt')}', 'w', encoding='utf-8') as f: f.write(html_to_txt(f'{read_dir}/{filename}'))

In [85]:
# fix these manually
l,l1,l2,l3,l4 = [],[],[],[],[]
for filename in os.listdir(write_dir):
    with open(f'{write_dir}/{filename}', encoding='utf-8') as f:
        t = f.read()
        if t.count("credits")>0: l.append(filename)
        if t.count(" ]")>0: l1.append(filename)
        if t.count("[\n")>0: l2.append(filename)
        if t.count("\n]")>0: l2.append(filename)
        if t.count(" ]")>0: l2.append(filename)
# remove text between start token and episode name: s10e1718
l,l1,l2,l3,l4

([], ['1004.txt', '1007.txt'], ['1004.txt', '1007.txt'], [], [])

In [86]:
d = {}
for filename in os.listdir(write_dir):
    with open(f'{write_dir}/{filename}', encoding='utf-8') as f: 
        lines = f.read().split('\n')
        m = 0
        l = None
        for line in lines:
            c = line.count(':')
            if c>m: 
                m = c
                l = line
        d[filename] = (m, line)
sorted(d.items(), key=lambda item: item[1], reverse=True)[:20]

[('0819.txt', (7, '<EPISODE_END>')),
 ('0208.txt', (6, '<EPISODE_END>')),
 ('0620.txt', (6, '<EPISODE_END>')),
 ('0112.txt', (4, '<EPISODE_END>')),
 ('0118.txt', (4, '<EPISODE_END>')),
 ('0510.txt', (4, '<EPISODE_END>')),
 ('1011.txt', (4, '<EPISODE_END>')),
 ('0316.txt', (3, '<EPISODE_END>')),
 ('0511.txt', (3, '<EPISODE_END>')),
 ('0523.txt', (3, '<EPISODE_END>')),
 ('0612.txt', (3, '<EPISODE_END>')),
 ('0801.txt', (3, '<EPISODE_END>')),
 ('0906.txt', (3, '<EPISODE_END>')),
 ('0919.txt', (3, '<EPISODE_END>')),
 ('0920.txt', (3, '<EPISODE_END>')),
 ('0101.txt', (2, '<EPISODE_END>')),
 ('0102.txt', (2, '<EPISODE_END>')),
 ('0103.txt', (2, '<EPISODE_END>')),
 ('0104.txt', (2, '<EPISODE_END>')),
 ('0106.txt', (2, '<EPISODE_END>'))]

In [87]:
outfilename = './data/interim/all.txt'
with open(outfilename, 'w', encoding='utf-8') as outfile:
    for filename in os.listdir(write_dir):
        if filename != 'all.txt':
            with open(f'{write_dir}/{filename}', encoding='utf-8') as infile:
                outfile.write(infile.read())
                outfile.write('\n')